# Indicators to SVI

In [1]:
# libraries
# Load R libraries
 if(!require("pacman"))
     install.packages("pacman")
     library("pacman")

p_load("dplyr", "sf", "purrr", "tidyverse", "terra", "corrplot")

print("Loaded Packages:")
p_loaded()

Loading required package: pacman



[1] "Loaded Packages:"


[1] "corrplot"  "terra"     "lubridate" "forcats"   "stringr"   "readr"    
 [7] "tidyr"     "tibble"    "ggplot2"   "tidyverse" "purrr"     "sf"       
[13] "dplyr"     "pacman"

In [2]:
# Load Data

# logrono census
logCensus <- read.csv("../../2_pipeline/Logrono/1a_CensusData/2021/census_oa_z_data_log.csv")

#get pop data
logCensusPop <- read.csv("../../2_pipeline/Logrono/1a_CensusData/2021/census_oa_raw_data_log.csv")
logCensusPop <- subset(logCensusPop, select = c('CUSEC', 'pop'))

# national census
nationalCensus <- read.csv("../../2_pipeline/Logrono/1a_CensusData/2021/persons_oa_z_data.csv")

#get pop data
nationalCensusPop <- read.csv("../../2_pipeline/Logrono/1a_CensusData/2021/persons_oa_raw.csv")
nationalCensusPop <- subset(nationalCensusPop, select = c('CUSEC', 'pop'))

# coperncius
tcd <- st_read("../../2_pipeline/Logrono/1b_Copernicus/2021/oa_TCD.geojson")
imd <- st_read("../../2_pipeline/Logrono/1b_Copernicus/2021/oa_IMD.geojson")

# get just spatial data 
oa <- subset(tcd, select = c('CUSEC', 'geometry'))

Reading layer `oa_TCD' from data source 
  `/Cities/2_pipeline/Logrono/1b_Copernicus/2021/oa_TCD.geojson' 
  using driver `GeoJSON'
Simple feature collection with 36333 features and 21 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -1004502 ymin: 3132130 xmax: 1126932 ymax: 4859240
Projected CRS: ETRS89 / UTM zone 30N
Reading layer `oa_IMD' from data source 
  `/Cities/2_pipeline/Logrono/1b_Copernicus/2021/oa_IMD.geojson' 
  using driver `GeoJSON'
Simple feature collection with 36333 features and 21 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -1004502 ymin: 3132130 xmax: 1126932 ymax: 4859240
Projected CRS: ETRS89 / UTM zone 30N


In [3]:
# need to pad the CUSEC with a leading 0
nationalCensus$CUSEC <- str_pad(nationalCensus$CUSEC, 10, pad = "0")
nationalCensusPop$CUSEC <- str_pad(nationalCensusPop$CUSEC, 10, pad = "0")

## Population Density

In [4]:
# caluclate population density - Log
logCensusGeom <- merge(oa,logCensus, by = "CUSEC")
logCensusGeom <- merge(logCensusGeom, logCensusPop,by = "CUSEC" )
logCensusGeom$area <- st_area(logCensusGeom)
logCensusGeom$popDen <- logCensusGeom$pop/logCensusGeom$area
logCensusGeom$popDen_Z <- scale(logCensusGeom$popDen)

#master dataset
logCensusGeom <- logCensusGeom %>% select(-c('pop', 'area', 'popDen'))

In [5]:
nrow(logCensusGeom)

[1] 343

In [6]:
# caluclate population density - Log
nationalCensusGeom <- merge(oa,nationalCensus, by = "CUSEC")
nationalCensusGeom <- merge(nationalCensusGeom, nationalCensusPop,by = "CUSEC" )
nationalCensusGeom$area <- st_area(nationalCensusGeom)
nationalCensusGeom$popDen <- nationalCensusGeom$pop/nationalCensusGeom$area
nationalCensusGeom$popDen_Z <- scale(nationalCensusGeom$popDen)

#master dataset
nationalCensusGeom <- nationalCensusGeom %>% select(-c('pop', 'area', 'popDen'))

In [7]:
nrow(nationalCensusGeom)

[1] 343

## Correlations

In [8]:
# check the correlations
correlationLogrono <- cor(st_drop_geometry(logCensusGeom %>% select(-c('CUSEC'))), use="complete.obs")
correlationNational <- cor(st_drop_geometry(nationalCensusGeom %>% select(-c('CUSEC'))), use="complete.obs")
correlationLogrono
correlationNational

,c_ab0t4_pct_Z,c_ag0t4_pct_Z,c_amo75_pct_Z,c_afo75_pct_Z,t13_1_pct_Z,t4_1_pct_Z,fam_3plus_pct_Z,invert_t9_1_pct_Z,t8_1_pct_Z,t10_1_pct_Z,c_ac5t9_pct_Z,t22_1_pct_Z,t5_1_pct_Z,t20_2_pct_Z,fam_bb_1970_pct_Z,popDen_Z
c_ab0t4_pct_Z,1.00000000,0.822964651,-0.50712792,-0.48283021,-0.12911707,0.68645620,0.05270976,-0.1375798,-0.25771365,-0.1686584,0.77012929,0.076224714,0.05320307,0.11171036,-0.1930935,-0.34426529
c_ag0t4_pct_Z,0.82296465,1.000000000,-0.55758644,-0.50884298,-0.13255587,0.70955453,0.10423487,-0.1584280,-0.16723645,-0.1610938,0.80405115,-0.005781832,0.06017842,0.09370702,-0.1871099,-0.35408171
c_amo75_pct_Z,-0.50712792,-0.557586436,1.00000000,0.90574414,0.00526875,-0.75848218,-0.07571057,0.2735870,-0.21649718,0.2812370,-0.68753441,0.340543671,0.21231973,0.10135840,0.3564873,0.57126120
c_afo75_pct_Z,-0.48283021,-0.508842981,0.90574414,1.00000000,0.08354839,-0.73299380,-0.08428225,0.2632148,-0.26903054,0.3215569,-0.66095769,0.496072161,0.23837916,0.24421892,0.5168301,0.54786812
t13_1_pct_Z,-0.12911707,-0.132555873,0.00526875,0.08354839,1.00000000,-0.15180110,0.08278737,0.5144936,-0.41052118,0.5761775,-0.17662233,0.240151627,0.33999675,0.32230530,0.2783139,0.15244876
t4_1_pct_Z,0.68645620,0.709554529,-0.75848218,-0.73299380,-0.15180110,1.00000000,0.12187587,-0.3053842,-0.02091793,-0.3683803,0.93438420,-0.267134305,-0.16414513,-0.11491318,-0.3322864,-0.52166907
fam_3plus_pct_Z,0.05270976,0.104234874,-0.07571057,-0.08428225,0.08278737,0.12187587,1.00000000,0.2543142,-0.04673474,0.3283595,0.09060817,-0.119517123,0.41213899,0.15333350,0.2783727,0.09608117
invert_t9_1_pct_Z,-0.13757983,-0.158428011,0.27358699,0.26321483,0.51449362,-0.30538417,0.25431419,1.0000000,-0.66278820,0.8477389,-0.30917867,0.118691358,0.83365059,0.30137298,0.4050462,0.54236263
t8_1_pct_Z,-0.25771365,-0.167236448,-0.21649718,-0.26903054,-0.41052118,-0.02091793,-0.04673474,-0.6627882,1.00000000,-0.6186627,-0.09340904,-0.466093448,-0.66010027,-0.42249634,-0.4033643,-0.35073319
t10_1_pct_Z,-0.16865836,-0.161093837,0.28123704,0.32155689,0.57617751,-0.36838026,0.32835951,0.8477389,-0.61866269,1.0000000,-0.34788113,0.330798936,0.80177313,0.50384168,0.5307593,0.54914578


,c_ab0t4_pct_Z,c_ag0t4_pct_Z,c_amo75_pct_Z,c_afo75_pct_Z,t13_1_pct_Z,t4_1_pct_Z,fam_3plus_pct_Z,invert_t9_1_pct_Z,t8_1_pct_Z,t10_1_pct_Z,c_ac5t9_pct_Z,t22_1_pct_Z,t5_1_pct_Z,t20_2_pct_Z,fam_bb_1970_pct_Z,popDen_Z
c_ab0t4_pct_Z,1.00000000,0.822964651,-0.50712792,-0.48283021,-0.12911707,0.68645620,0.05270976,-0.1375798,-0.25771365,-0.1686584,0.77012929,0.076224714,0.05320307,0.11171036,-0.1930935,-0.34426529
c_ag0t4_pct_Z,0.82296465,1.000000000,-0.55758644,-0.50884298,-0.13255587,0.70955453,0.10423487,-0.1584280,-0.16723645,-0.1610938,0.80405115,-0.005781832,0.06017842,0.09370702,-0.1871099,-0.35408171
c_amo75_pct_Z,-0.50712792,-0.557586436,1.00000000,0.90574414,0.00526875,-0.75848218,-0.07571057,0.2735870,-0.21649718,0.2812370,-0.68753441,0.340543671,0.21231973,0.10135840,0.3564873,0.57126120
c_afo75_pct_Z,-0.48283021,-0.508842981,0.90574414,1.00000000,0.08354839,-0.73299380,-0.08428225,0.2632148,-0.26903054,0.3215569,-0.66095769,0.496072161,0.23837916,0.24421892,0.5168301,0.54786812
t13_1_pct_Z,-0.12911707,-0.132555873,0.00526875,0.08354839,1.00000000,-0.15180110,0.08278737,0.5144936,-0.41052118,0.5761775,-0.17662233,0.240151627,0.33999675,0.32230530,0.2783139,0.15244876
t4_1_pct_Z,0.68645620,0.709554529,-0.75848218,-0.73299380,-0.15180110,1.00000000,0.12187587,-0.3053842,-0.02091793,-0.3683803,0.93438420,-0.267134305,-0.16414513,-0.11491318,-0.3322864,-0.52166907
fam_3plus_pct_Z,0.05270976,0.104234874,-0.07571057,-0.08428225,0.08278737,0.12187587,1.00000000,0.2543142,-0.04673474,0.3283595,0.09060817,-0.119517123,0.41213899,0.15333350,0.2783727,0.09608117
invert_t9_1_pct_Z,-0.13757983,-0.158428011,0.27358699,0.26321483,0.51449362,-0.30538417,0.25431419,1.0000000,-0.66278820,0.8477389,-0.30917867,0.118691358,0.83365059,0.30137298,0.4050462,0.54236263
t8_1_pct_Z,-0.25771365,-0.167236448,-0.21649718,-0.26903054,-0.41052118,-0.02091793,-0.04673474,-0.6627882,1.00000000,-0.6186627,-0.09340904,-0.466093448,-0.66010027,-0.42249634,-0.4033643,-0.35073319
t10_1_pct_Z,-0.16865836,-0.161093837,0.28123704,0.32155689,0.57617751,-0.36838026,0.32835951,0.8477389,-0.61866269,1.0000000,-0.34788113,0.330798936,0.80177313,0.50384168,0.5307593,0.54914578


In [9]:
#TEMP - no household data
## homes owned and renting are well correlated - remove homes owned data
#logCensusGeom <- logCensusGeom  %>% select(-c('homesOwned_pct_Z'))
#nationalCensusGeom <- nationalCensusGeom %>% select(-c('homesOwned'))

# remove population density as its difficult to justify for social vulnerability - more like enhanced exposure covered by Cop
logCensusGeom <- logCensusGeom  %>% select(-c('popDen_Z'))
nationalCensusGeom <- nationalCensusGeom %>% select(-c('popDen_Z'))

## Logrono

In [10]:
# update code, remove this block and rewrite 2021 code underneath

## combine vulnerability score - Logrono
#logCensusGeom$vuln <- rowMeans(st_drop_geometry(logCensusGeom)[,2:8], na.rm=TRUE)
#
## some cells have NA data - count how many
#logCensusGeom$na_count <- apply(logCensusGeom, 1, function(x) sum(is.na(x)))

cols <- ncol(st_drop_geometry(logCensusGeom))
logCensusNoGeom <- st_drop_geometry(logCensusGeom)[,2:cols]
#head(logCensusNoGeom)

##############
### Domain ###
##############

### Domain - Age 
ageIndicators <- subset(logCensusNoGeom, select=c('c_ab0t4_pct_Z', 'c_ag0t4_pct_Z', 'c_amo75_pct_Z', 'c_afo75_pct_Z'))
logCensusGeom$ageDomain <- rowMeans(ageIndicators, na.rm=TRUE)
#head(ageIndicators)
#head(logCensusGeom$ageDomain)

### Domain - Health
healthIndicators <- subset(logCensusNoGeom, select=c('t13_1_pct_Z'))
logCensusGeom$healthDomain <- rowMeans(healthIndicators, na.rm=TRUE)
#head(healthIndicators)
#head(logCensusGeom$healthDomain)

### Domain - Mobility
mobilityIndicators <- subset(logCensusNoGeom, select=c('t4_1_pct_Z', 'fam_3plus_pct_Z'))
logCensusGeom$mobilityDomain <- rowMeans(mobilityIndicators, na.rm=TRUE)
#head(mobilityIndicators)
#head(logCensusGeom$mobilityDomain)

### Domain - Income
incomeIndicators <- subset(logCensusNoGeom, select=c('invert_t9_1_pct_Z', 't8_1_pct_Z', 't10_1_pct_Z'))
logCensusGeom$incomeDomain <- rowMeans(incomeIndicators, na.rm=TRUE)
#head(incomeIndicators)
#head(logCensusGeom$incomeDomain)

### Domain - Social Network
socialNetworkIndicators <- subset(logCensusNoGeom, select=c('c_ac5t9_pct_Z', 't22_1_pct_Z'))
logCensusGeom$socialNetworkDomain <- rowMeans(socialNetworkIndicators, na.rm=TRUE)
#head(socialNetworkIndicators)
#head(logCensusGeom$socialNetworkDomain)

### Domain - Local Knowledge
localKnowledgeIndicators <- subset(logCensusNoGeom, select=c('t5_1_pct_Z'))
logCensusGeom$localKnowledgeDomain <- rowMeans(localKnowledgeIndicators, na.rm=TRUE)
#head(localKnowledgeIndicators)
#head(logCensusGeom$localKnowledgeDomain)

### Domain - Tenure
tenureIndicators <- subset(logCensusNoGeom, select=c('t20_2_pct_Z'))
logCensusGeom$tenureDomain <- rowMeans(tenureIndicators, na.rm=TRUE)
#head(tenureIndicators)
#head(logCensusGeom$tenureDomain)

#################
### Dimension ###
#################

# Dimension - Sensitivity
sensitivityDomains <- cbind(logCensusGeom$ageDomain,
                            logCensusGeom$healthDomain)
logCensusGeom$sensitivityDimension <- rowMeans(sensitivityDomains, na.rm=TRUE)
#head(sensitivityDomains)
#head(logCensusGeom$sensitivityDimension)

# Dimension - Adaptive Capacity
adaptiveCapacityDomains <- cbind(logCensusGeom$mobilityDomain,
                                 logCensusGeom$incomeDomain,
                                 logCensusGeom$socialNetworkDomain,
                                 logCensusGeom$localKnowledgeDomain,
                                 logCensusGeom$tenureDomain)
logCensusGeom$adaptiveCapacityDimension <- rowMeans(adaptiveCapacityDomains, na.rm=TRUE)
#head(adaptiveCapacityDomains)
#head(logCensusGeom$adaptiveCapacityDimension)


# some cells have NA data - count how many
logCensusGeom$na_count <- apply(logCensusGeom, 1, function(x) sum(is.na(x)))
#head(logCensusGeom)                                

## National

In [11]:
# TO DO, similar method as Logrono above

## combine vulnerability score - National
#nationalCensusGeom$vuln <- rowMeans(st_drop_geometry(nationalCensusGeom)[,2:8], na.rm=TRUE)
#
## some cells have NA data - count how many
#nationalCensusGeom$na_count <- apply(nationalCensusGeom, 1, function(x) sum(is.na(x)))                                

In [12]:
nrow(nationalCensusGeom)

[1] 343

## Enhanced Exposure (including Copernicus)

In [13]:
# combine vulnerability score - Logrono with Copernicus
logCensusCop <- merge(tcd,st_drop_geometry(logCensusGeom), by= "CUSEC")
logCensusCop <- merge(imd,st_drop_geometry(logCensusCop), by= "CUSEC")

# # calculate Copernicus Z score
logCensusCop$tcdZ <- -scale(logCensusCop$tcdPerc)
logCensusCop$impZ <- scale(logCensusCop$impPerc)

# # trim the columns
logCensusCop <- subset(logCensusCop, select=c(names(logCensusGeom[,1:cols]),'tcdZ', 'impZ'))
# logCensusCop <- logCensusCop %>% select(-c('vuln', 'na_count'))

# new 2021 code
logCensusGeomWithCop <- logCensusGeom
logCensusGeomWithCop$tcdZ <- logCensusCop$tcdZ
logCensusGeomWithCop$impZ <- logCensusCop$impZ

###############
### Domains ###
###############

### Domain - Physical Environment
physicalEnvironmentIndicators <- cbind(logCensusGeomWithCop$tcdZ,
                                       logCensusGeomWithCop$impZ)
logCensusGeomWithCop$physicalEnvironmentDomain <- rowMeans(physicalEnvironmentIndicators, na.rm=TRUE)
#head(physicalEnvironmentIndicators)
#head(logCensusGeomWithCop$physicalEnvironmentDomain)

### Domain - Housing Characteristics
housingCharacteristicsIndicators <- cbind(logCensusGeomWithCop$fam_bb_1970_pct_Z)
logCensusGeomWithCop$housingCharacteristicsDomain <- rowMeans(housingCharacteristicsIndicators, na.rm=TRUE)
#head(housingCharacteristicsIndicators)
#head(logCensusGeomWithCop$housingCharacteristicsDomain)

#################
### Dimension ###
#################

# Dimension - Enhanced Exposure
enhancedExposureDomains <- cbind(logCensusGeomWithCop$physicalEnvironmentDomain,
                                 logCensusGeomWithCop$housingCharacteristicsDomain)
logCensusGeomWithCop$enhancedExposureDimension <- rowMeans(enhancedExposureDomains, na.rm=TRUE)
#head(enhancedExposureDomains)
#head(logCensusGeomWithCop$enhancedExposureDimension)

#head(logCensusGeom)
#head(logCensusGeomWithCop)

In [14]:
correlationLogronoCop <- cor(st_drop_geometry(logCensusCop %>% select(-c('CUSEC'))), use="complete.obs")
correlationLogronoCop

,c_ab0t4_pct_Z,c_ag0t4_pct_Z,c_amo75_pct_Z,c_afo75_pct_Z,t13_1_pct_Z,t4_1_pct_Z,fam_3plus_pct_Z,invert_t9_1_pct_Z,t8_1_pct_Z,t10_1_pct_Z,c_ac5t9_pct_Z,t22_1_pct_Z,t5_1_pct_Z,t20_2_pct_Z,fam_bb_1970_pct_Z,tcdZ,impZ
c_ab0t4_pct_Z,1.00000000,0.822964651,-0.50712792,-0.48283021,-0.12911707,0.68645620,0.05270976,-0.1375798,-0.25771365,-0.1686584,0.77012929,0.076224714,0.05320307,0.11171036,-0.1930935,0.11947105,-0.35046105
c_ag0t4_pct_Z,0.82296465,1.000000000,-0.55758644,-0.50884298,-0.13255587,0.70955453,0.10423487,-0.1584280,-0.16723645,-0.1610938,0.80405115,-0.005781832,0.06017842,0.09370702,-0.1871099,0.06606116,-0.34493508
c_amo75_pct_Z,-0.50712792,-0.557586436,1.00000000,0.90574414,0.00526875,-0.75848218,-0.07571057,0.2735870,-0.21649718,0.2812370,-0.68753441,0.340543671,0.21231973,0.10135840,0.3564873,0.14774747,0.59434333
c_afo75_pct_Z,-0.48283021,-0.508842981,0.90574414,1.00000000,0.08354839,-0.73299380,-0.08428225,0.2632148,-0.26903054,0.3215569,-0.66095769,0.496072161,0.23837916,0.24421892,0.5168301,0.22618073,0.65393169
t13_1_pct_Z,-0.12911707,-0.132555873,0.00526875,0.08354839,1.00000000,-0.15180110,0.08278737,0.5144936,-0.41052118,0.5761775,-0.17662233,0.240151627,0.33999675,0.32230530,0.2783139,0.04841157,0.16679604
t4_1_pct_Z,0.68645620,0.709554529,-0.75848218,-0.73299380,-0.15180110,1.00000000,0.12187587,-0.3053842,-0.02091793,-0.3683803,0.93438420,-0.267134305,-0.16414513,-0.11491318,-0.3322864,0.01088699,-0.54016323
fam_3plus_pct_Z,0.05270976,0.104234874,-0.07571057,-0.08428225,0.08278737,0.12187587,1.00000000,0.2543142,-0.04673474,0.3283595,0.09060817,-0.119517123,0.41213899,0.15333350,0.2783727,0.09870252,0.08382116
invert_t9_1_pct_Z,-0.13757983,-0.158428011,0.27358699,0.26321483,0.51449362,-0.30538417,0.25431419,1.0000000,-0.66278820,0.8477389,-0.30917867,0.118691358,0.83365059,0.30137298,0.4050462,0.23184465,0.38097524
t8_1_pct_Z,-0.25771365,-0.167236448,-0.21649718,-0.26903054,-0.41052118,-0.02091793,-0.04673474,-0.6627882,1.00000000,-0.6186627,-0.09340904,-0.466093448,-0.66010027,-0.42249634,-0.4033643,-0.24393253,-0.29671264
t10_1_pct_Z,-0.16865836,-0.161093837,0.28123704,0.32155689,0.57617751,-0.36838026,0.32835951,0.8477389,-0.61866269,1.0000000,-0.34788113,0.330798936,0.80177313,0.50384168,0.5307593,0.16829620,0.49046730


In [15]:
##calculate vulnerability with COP
#logCensusCop$vulnCop <- rowMeans(st_drop_geometry(logCensusCop[,2:11]), na.rm=TRUE)
#logCensusCop$na_count <- apply(logCensusCop, 1, function(x) sum(is.na(x)))
#head(logCensusCop)

#2021 code
# Vulnerability WITH enhancedExposureDimension
vulnerabilityInputs1 <- cbind(logCensusGeom$sensitivityDimension,
                              logCensusGeom$adaptiveCapacityDimension)
logCensusGeom$vulnerability <- rowMeans(vulnerabilityInputs1, na.rm=TRUE)

# Vulnerability WITHOUT enhancedExposureDimension
vulnerabilityInputs2 <- cbind(logCensusGeomWithCop$sensitivityDimension,
                              logCensusGeomWithCop$adaptiveCapacityDimension,
                              logCensusGeomWithCop$enhancedExposureDimension)
logCensusGeomWithCop$vulnerability <- rowMeans(vulnerabilityInputs2, na.rm=TRUE)

head(logCensusGeom)
head(logCensusGeomWithCop)

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



CUSEC      c_ab0t4_pct_Z c_ag0t4_pct_Z c_amo75_pct_Z c_afo75_pct_Z
1 2600101001 -0.19258164   -0.1467916     0.4818082    -0.10021618  
2 2600201001  0.23745377    0.3146154    -0.2611307    -0.31519260  
3 2600301001 -0.96122760   -0.1140934     0.3255496     0.83585043  
4 2600401001  0.01083739   -1.4002199     0.5976241    -1.28188374  
5 2600501001 -0.10396496    0.1640514    -0.1893316     0.06098884  
6 2600502001  2.13595981    1.4136596    -0.7006675    -0.72507994  
  t13_1_pct_Z t4_1_pct_Z   fam_3plus_pct_Z invert_t9_1_pct_Z t8_1_pct_Z 
1  1.15716086 -0.580972960 NA              0.5991024          1.07598605
2  0.09800059 -0.192274385 NA              0.8812495         -0.06928698
3 -0.96115967 -0.580972960 NA              0.7872005          0.50334953
4          NA           NA NA                     NA                  NA
5  1.15716086  0.002074903 NA              0.8812495         -0.64192350
6  0.09800059  1.556869207 NA              0.4110043         -0.06928698
  geometry                       ⋯ healthDomain mobilityDomain incomeDomain
1 MULTIPOLYGON (((526123.5 47... ⋯  1.15716086  -0.580972960   -0.07901436 
2 MULTIPOLYGON (((556420.1 47... ⋯  0.09800059  -0.192274385    0.51015822 
3 MULTIPOLYGON (((585309.2 46... ⋯ -0.96115967  -0.580972960    0.08509990 
4 MULTIPOLYGON (((552619.3 46... ⋯         NaN           NaN           NaN 
5 MULTIPOLYGON (((542645.6 46... ⋯  1.15716086   0.002074903    0.46542625 
6 MULTIPOLYGON (((543287.7 46... ⋯  0.09800059   1.556869207   -0.08503079 
  socialNetworkDomain localKnowledgeDomain tenureDomain sensitivityDimension
1  0.6457650          -0.1471302            0.3976581    0.58385779         
2  0.2092563           0.5632406           -0.2843347    0.04596853         
3  0.4780322           0.1370181           -0.6560445   -0.46981996         
4 -1.5705327                 NaN                  NaN   -0.51841053         
5 -0.2064757           0.4211664           -0.4370448    0.57004838         
6  0.3109133          -0.1471302           -0.6100780    0.31448430         
  adaptiveCapacityDimension na_count vulnerability
1  0.04726112                2        0.3155595   
2  0.16120920                2        0.1035889   
3 -0.10737344                2       -0.2885967   
4 -1.57053273               15       -1.0444716   
5  0.04902942                2        0.3095389   
6  0.20510872                2        0.2597965   
  geometry                      
1 MULTIPOLYGON (((526123.5 47...
2 MULTIPOLYGON (((556420.1 47...
3 MULTIPOLYGON (((585309.2 46...
4 MULTIPOLYGON (((552619.3 46...
5 MULTIPOLYGON (((542645.6 46...
6 MULTIPOLYGON (((543287.7 46...

CUSEC      c_ab0t4_pct_Z c_ag0t4_pct_Z c_amo75_pct_Z c_afo75_pct_Z
1 2600101001 -0.19258164   -0.1467916     0.4818082    -0.10021618  
2 2600201001  0.23745377    0.3146154    -0.2611307    -0.31519260  
3 2600301001 -0.96122760   -0.1140934     0.3255496     0.83585043  
4 2600401001  0.01083739   -1.4002199     0.5976241    -1.28188374  
5 2600501001 -0.10396496    0.1640514    -0.1893316     0.06098884  
6 2600502001  2.13595981    1.4136596    -0.7006675    -0.72507994  
  t13_1_pct_Z t4_1_pct_Z   fam_3plus_pct_Z invert_t9_1_pct_Z t8_1_pct_Z 
1  1.15716086 -0.580972960 NA              0.5991024          1.07598605
2  0.09800059 -0.192274385 NA              0.8812495         -0.06928698
3 -0.96115967 -0.580972960 NA              0.7872005          0.50334953
4          NA           NA NA                     NA                  NA
5  1.15716086  0.002074903 NA              0.8812495         -0.64192350
6  0.09800059  1.556869207 NA              0.4110043         -0.06928698
  geometry                       ⋯ tenureDomain sensitivityDimension
1 MULTIPOLYGON (((526123.5 47... ⋯  0.3976581    0.58385779         
2 MULTIPOLYGON (((556420.1 47... ⋯ -0.2843347    0.04596853         
3 MULTIPOLYGON (((585309.2 46... ⋯ -0.6560445   -0.46981996         
4 MULTIPOLYGON (((552619.3 46... ⋯        NaN   -0.51841053         
5 MULTIPOLYGON (((542645.6 46... ⋯ -0.4370448    0.57004838         
6 MULTIPOLYGON (((543287.7 46... ⋯ -0.6100780    0.31448430         
  adaptiveCapacityDimension na_count tcdZ       impZ      
1  0.04726112                2       -0.2544957 -0.7508389
2  0.16120920                2        0.5137644 -0.5983790
3 -0.10737344                2        0.2432283 -0.7776366
4 -1.57053273               15       -2.0079509 -0.7839835
5  0.04902942                2       -0.3397642 -0.6345343
6  0.20510872                2       -1.0923204 -0.6383858
  physicalEnvironmentDomain housingCharacteristicsDomain
1 -0.50266730               NaN                         
2 -0.04230734               NaN                         
3 -0.26720417               NaN                         
4 -1.39596719               NaN                         
5 -0.48714929               NaN                         
6 -0.86535311               NaN                         
  enhancedExposureDimension vulnerability geometry                      
1 -0.50266730                0.04281720   MULTIPOLYGON (((526123.5 47...
2 -0.04230734                0.05495679   MULTIPOLYGON (((556420.1 47...
3 -0.26720417               -0.28146586   MULTIPOLYGON (((585309.2 46...
4 -1.39596719               -1.16163682   MULTIPOLYGON (((552619.3 46...
5 -0.48714929                0.04397617   MULTIPOLYGON (((542645.6 46...
6 -0.86535311               -0.11525336   MULTIPOLYGON (((543287.7 46...

## National with Copernicus

In [16]:
# TO DO, similar method as Logrono above

## combine vulnerability score - Logrono with Copernicus
#nationalCensusCop <- merge(tcd,st_drop_geometry(nationalCensusGeom), by= "CUSEC")
#nationalCensusCop <- merge(imd,st_drop_geometry(nationalCensusCop), by= "CUSEC")
#
## # calculate Copernicus Z score
#nationalCensusCop$tcdZ <- -scale(nationalCensusCop$tcdPerc)
#nationalCensusCop$impZ <- scale(nationalCensusCop$impPerc)
#
## trim the columns
#nationalCensusCop <- subset(nationalCensusCop, select=c(names(nationalCensusGeom[,1:8]),'tcdZ', 'impZ'))
## nationalCensusCop <- nationalCensusCop %>% select(-c('vuln', 'na_count'))

In [17]:
#correlationNationalCop <- cor(st_drop_geometry(nationalCensusCop %>% select(-c('CUSEC'))), use="complete.obs")
#correlationNationalCop

In [18]:
##calculate vulnerability with COP
#nationalCensusCop$vulnCop <- rowMeans(st_drop_geometry(nationalCensusCop[,2:11]), na.rm=TRUE)
#
#nationalCensusCop$na_count <- apply(nationalCensusCop, 1, function(x) sum(is.na(x)))
##nationalCensusCop

## Export

In [19]:
# create the output directory if it does not exist
outputDir <- file.path("../..","3_outputs","Logrono","2021")
if(!dir.exists(outputDir)){
    dir.create(outputDir, recursive = TRUE)
    print(paste0(outputDir, " created"))
}

# GeoJSON
st_write(logCensusGeom, file.path(outputDir,"logrono_vulnerability_2021_without_copernicus.geojson"), delete_dsn=TRUE)
st_write(logCensusGeomWithCop, file.path(outputDir,"logrono_vulnerability_2021_with_copernicus.geojson"), delete_dsn=TRUE)
#st_write(nationalCensusGeom, file.path(outputDir,"national_vulnerability_2021.geojson"), delete_dsn=TRUE)
#st_write(logCensusCop, file.path(outputDir,"logrono_copernicus_vulnerability_2021.geojson"), delete_dsn=TRUE)
#st_write(nationalCensusCop, file.path(outputDir,"national_copernicus_vulnerability_2021.geojson"), delete_dsn=TRUE)

# Shapefile
st_write(logCensusGeom, file.path(outputDir,"logrono_vulnerability_2021_without_copernicus.shp"), append = FALSE)
st_write(logCensusGeomWithCop, file.path(outputDir,"logrono_vulnerability_2021_with_copernicus.shp"), append = FALSE)
#st_write(nationalCensusGeom, file.path(outputDir,"national_vulnerability_2021.shp"), append = FALSE)
#st_write(logCensusCop, file.path(outputDir,"logrono_copernicus_vulnerability_2021.shp"), append = FALSE)
#st_write(nationalCensusCop, file.path(outputDir,"national_copernicus_vulnerability_2021.shp"), append = FALSE)

Deleting source `../../3_outputs/Logrono/2021/logrono_vulnerability_2021_without_copernicus.geojson' using driver `GeoJSON'
Writing layer `logrono_vulnerability_2021_without_copernicus' to data source 
  `../../3_outputs/Logrono/2021/logrono_vulnerability_2021_without_copernicus.geojson' using driver `GeoJSON'
Writing 343 features with 27 fields and geometry type Multi Polygon.
Deleting source `../../3_outputs/Logrono/2021/logrono_vulnerability_2021_with_copernicus.geojson' using driver `GeoJSON'
Writing layer `logrono_vulnerability_2021_with_copernicus' to data source 
  `../../3_outputs/Logrono/2021/logrono_vulnerability_2021_with_copernicus.geojson' using driver `GeoJSON'
Writing 343 features with 32 fields and geometry type Multi Polygon.


Warning message in abbreviate_shapefile_names(obj):
“Field names abbreviated for ESRI Shapefile driver”


Deleting layer `logrono_vulnerability_2021_without_copernicus' using driver `ESRI Shapefile'
Writing layer `logrono_vulnerability_2021_without_copernicus' to data source 
  `../../3_outputs/Logrono/2021/logrono_vulnerability_2021_without_copernicus.shp' using driver `ESRI Shapefile'
Writing 343 features with 27 fields and geometry type Multi Polygon.


Warning message in abbreviate_shapefile_names(obj):
“Field names abbreviated for ESRI Shapefile driver”


Deleting layer `logrono_vulnerability_2021_with_copernicus' using driver `ESRI Shapefile'
Writing layer `logrono_vulnerability_2021_with_copernicus' to data source 
  `../../3_outputs/Logrono/2021/logrono_vulnerability_2021_with_copernicus.shp' using driver `ESRI Shapefile'
Writing 343 features with 32 fields and geometry type Multi Polygon.


**END**